In [1]:
import numpy as np
import os
from tqdm.auto import tqdm
# from pathos.multiprocessing import ProcessingPool as Pool
# from functools import partial
import pandas as pd
from PIL import Image

In [2]:
# clone Scaled_YOLOv4
!git clone https://github.com/tom-auger/ScaledYOLOv4.git
%cd ScaledYOLOv4/
#checkout the yolov4-large branch
!git checkout yolov4-large

Cloning into 'ScaledYOLOv4'...
remote: Enumerating objects: 42, done.
remote: Counting objects: 100% (42/42), done.
remote: Compressing objects: 100% (29/29), done.
remote: Total 170 (delta 28), reused 27 (delta 13), pack-reused 128
Receiving objects: 100% (170/170), 366.26 KiB | 1.40 MiB/s, done.
Resolving deltas: 100% (80/80), done.
/kaggle/working/ScaledYOLOv4
Branch 'yolov4-large' set up to track remote branch 'yolov4-large' from 'origin'.
Switched to a new branch 'yolov4-large'


In [3]:
# df_train = pd.read_csv('/kaggle/input/vinbigstratifiedyolo/train_orig.csv')
# df_val = pd.read_csv('/kaggle/input/vinbigstratifiedyolo/validation_orig.csv')
# df_test = pd.read_csv('/kaggle/input/vinbigstratifiedyolo/test_orig.csv')

In [4]:
import torch
print('Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

Using torch 1.7.0 _CudaDeviceProperties(name='Tesla P100-PCIE-16GB', major=6, minor=0, total_memory=16280MB, multi_processor_count=56)


In [5]:
#install mish activation funciton for cuda
%cd ..
!git clone https://github.com/JunnYu/mish-cuda
%cd mish-cuda
!python setup.py build install
%cd ..

/kaggle/working
Cloning into 'mish-cuda'...
remote: Enumerating objects: 88, done.
remote: Counting objects: 100% (88/88), done.
remote: Compressing objects: 100% (75/75), done.
remote: Total 195 (delta 7), reused 79 (delta 3), pack-reused 107
Receiving objects: 100% (195/195), 208.77 KiB | 841.00 KiB/s, done.
Resolving deltas: 100% (56/56), done.
/kaggle/working/mish-cuda
/opt/conda/lib/python3.7/distutils/extension.py:131: UserWarning: Unknown Extension options: 'headers'
  warnings.warn(msg)
running build
running build_py
creating build
creating build/lib.linux-x86_64-3.7
creating build/lib.linux-x86_64-3.7/mish_cuda
copying src/mish_cuda/__init__.py -> build/lib.linux-x86_64-3.7/mish_cuda
running egg_info
creating src/mish_cuda.egg-info
writing src/mish_cuda.egg-info/PKG-INFO
writing dependency_links to src/mish_cuda.egg-info/dependency_links.txt
writing requirements to src/mish_cuda.egg-info/requires.txt
writing top-level names to src/mish_cuda.egg-info/top_level.txt
writing manif

In [6]:
# abnormalities = ['Aortic enlargement', 'Atelectasis', 'Calcification', 'Cardiomegaly', 'Consolidation', 'ILD', 'Infiltration', 'Lung Opacity', 'Nodule/Mass', 'Other lesion', 'Pleural effusion', 'Pleural thickening', 'Pneumothorax', 'Pulmonary fibrosis']

# %mkdir data
# def write_to_file(imgs, dataset_name, ab_name):
#     with open(f"data/{dataset_name}_{ab_name}.txt", 'wt') as f:
#         for img in imgs:
#             f.write(f"/kaggle/input/mlpyoloall/{dataset_name}/{img}.png\n")
    
# def write_yaml_file(ab):
#     with open(f"data/{ab}.yaml", 'wt') as f:
#         f.write(f"""train: ../data/train_{ab}.txt
# val: ../data/validation_{ab}.txt

# nc: 14
# names: ['Aortic enlargement', 'Atelectasis', 'Calcification', 'Cardiomegaly', 'Consolidation', 'ILD', 'Infiltration', 'Lung Opacity', 'Nodule/Mass', 'Other lesion', 'Pleural effusion', 'Pleural thickening', 'Pneumothorax', 'Pulmonary fibrosis']
# """)
        
# for ab in abnormalities:
#     train_imgs = df_train[df_train['class_name'] == ab]['image_id'].unique()
#     val_imgs = df_val[df_val['class_name'] == ab]['image_id'].unique()
#     test_imgs = df_test[df_test['class_name'] == ab]['image_id'].unique()
    
#     # Write each list to a data spec file
#     ab = ab.replace('/', '_').replace(' ', '_')
#     write_to_file(train_imgs, 'train', ab)
#     write_to_file(val_imgs, 'validation', ab)
#     write_to_file(test_imgs, 'test', ab)
    
#     # Write the data yaml file
#     write_yaml_file(ab)

In [7]:
with open(f"data.yaml", 'wt') as f:
        f.write(f"""train: /kaggle/input/vinbigstratifiedyolo/vinbigstratified/train
val: /kaggle/input/vinbigstratifiedyolo/vinbigstratified/validation

nc: 14
names: ['Aortic enlargement', 'Atelectasis', 'Calcification', 'Cardiomegaly', 'Consolidation', 'ILD', 'Infiltration', 'Lung Opacity', 'Nodule/Mass', 'Other lesion', 'Pleural effusion', 'Pleural thickening', 'Pneumothorax', 'Pulmonary fibrosis']
""")

In [8]:
%cd ScaledYOLOv4/

/kaggle/working/ScaledYOLOv4


In [9]:
!python train.py --labels 'labels_all' --img 640 --batch 16 --epochs 20 --data '../data.yaml' --cfg ./models/yolov4-csp.yaml --weights '' --name yolov4-csp-results

Using CUDA device0 _CudaDeviceProperties(name='Tesla P100-PCIE-16GB', total_memory=16280MB)

Namespace(adam=False, batch_size=16, bucket='', cache_images=False, cfg='./models/yolov4-csp.yaml', data='../data.yaml', device='', epochs=20, evolve=False, global_rank=-1, hyp='data/hyp.scratch.yaml', img_size=[640, 640], labels='labels_all', local_rank=-1, logdir='runs/', multi_scale=False, name='yolov4-csp-results', noautoanchor=False, nosave=False, notest=False, onlyclass=-1, rect=False, resume=False, single_cls=False, sync_bn=False, total_batch_size=16, weights='', world_size=1)
Start Tensorboard with "tensorboard --logdir runs/", view at http://localhost:6006/
2021-03-28 04:20:01.127289: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.2
Hyperparameters {'lr0': 0.01, 'momentum': 0.937, 'weight_decay': 0.0005, 'giou': 0.05, 'cls': 0.5, 'cls_pw': 1.0, 'obj': 1.0, 'obj_pw': 1.0, 'iou_t': 0.2, 'anchor_t': 4.0, 'fl_gamma': 0.0,

In [10]:
%%bash
du -h /kaggle/working/ScaledYOLOv4/

16K	/kaggle/working/ScaledYOLOv4/data
96K	/kaggle/working/ScaledYOLOv4/utils/__pycache__
224K	/kaggle/working/ScaledYOLOv4/utils
16K	/kaggle/working/ScaledYOLOv4/__pycache__
48K	/kaggle/working/ScaledYOLOv4/models/__pycache__
116K	/kaggle/working/ScaledYOLOv4/models
8.5G	/kaggle/working/ScaledYOLOv4/runs/exp0_yolov4-csp-results/weights
8.5G	/kaggle/working/ScaledYOLOv4/runs/exp0_yolov4-csp-results
8.5G	/kaggle/working/ScaledYOLOv4/runs
4.0K	/kaggle/working/ScaledYOLOv4/.git/branches
8.0K	/kaggle/working/ScaledYOLOv4/.git/info
12K	/kaggle/working/ScaledYOLOv4/.git/logs/refs/heads
8.0K	/kaggle/working/ScaledYOLOv4/.git/logs/refs/remotes/origin
12K	/kaggle/working/ScaledYOLOv4/.git/logs/refs/remotes
28K	/kaggle/working/ScaledYOLOv4/.git/logs/refs
36K	/kaggle/working/ScaledYOLOv4/.git/logs
12K	/kaggle/working/ScaledYOLOv4/.git/refs/heads
4.0K	/kaggle/working/ScaledYOLOv4/.git/refs/tags
8.0K	/kaggle/working/ScaledYOLOv4/.git/refs/remotes/origin
12K	/kaggle/working/ScaledYOLOv4/.git/refs/rem